<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_omniversal_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#!/usr/bin/env python3
"""
train_omniversal_ai.py

Pipeline for OmniversalAI:
1. Synthetic “omniversal intelligence” dataset (6 inputs → 3 targets)
2. float32 normalization & dtype consistency
3. MLP with MC-Dropout for uncertainty
4. Physics-informed residual enforcing toy omniversal laws
5. Combined loss (MSE + physics)
6. Training loop: AdamW, scheduler, grad clipping, early stopping
7. Checkpointing & reload
8. Visualizations: loss curves, parity plot, uncertainty histogram
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt


# -----------------------------------------------------------------------------
# 1. Synthetic Omniversal Intelligence Dataset
# -----------------------------------------------------------------------------
class OmniversalDataset(Dataset):
    def __init__(self, n_samples=5000, seed=42):
        np.random.seed(seed)
        F = [np.random.uniform(-1, 1, (n_samples, 1)) for _ in range(6)]
        X_raw = np.hstack(F).astype(np.float64)
        f1, f2, f3, f4, f5, f6 = X_raw.T

        # toy ground‐truth metrics
        FR = np.sin(f1 * f2) + np.cos(f3)                   # field resonance
        IC = np.exp(-np.abs(f4 + f5)) + np.tanh(f6)         # intelligence coherence
        NBI = FR * IC                                       # non‐being integration

        Y_raw = np.stack([FR, IC, NBI], axis=1).astype(np.float64)
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # compute normalization stats
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-8
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-8

        # normalize and cast to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )


# -----------------------------------------------------------------------------
# 2. OmniversalAI Model with Dropout
# -----------------------------------------------------------------------------
class OmniversalAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, p_drop=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p_drop),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)


# -----------------------------------------------------------------------------
# 3. Physics‐Informed Residual Loss
# -----------------------------------------------------------------------------
def physics_residual(pred, X_norm, stats):
    X_den = X_norm * stats['X_std'] + stats['X_mean']
    f1, f2, f3, f4, f5, f6 = X_den.t()

    FR_t = torch.sin(f1 * f2) + torch.cos(f3)
    IC_t = torch.exp(-torch.abs(f4 + f5)) + torch.tanh(f6)
    NBI_t = FR_t * IC_t

    Yt = torch.stack([FR_t, IC_t, NBI_t], dim=1)
    Yt_n = (Yt - stats['Y_mean']) / stats['Y_std']

    return nn.MSELoss()(pred, Yt_n)


def total_loss(pred, true, X_norm, stats, lam=1.0):
    mse = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X_norm, stats)
    return mse + lam * phys, mse, phys


# -----------------------------------------------------------------------------
# 4. MC‐Dropout Uncertainty Estimation
# -----------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds)
    return arr.mean(0), arr.std(0)


# -----------------------------------------------------------------------------
# 5. Training Loop
# -----------------------------------------------------------------------------
def train(model, tr_dl, va_dl, stats, device,
          lr=1e-3, weight_decay=1e-5, lam=1.0,
          epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val = float('inf')
    wait = 0
    history = {'train': [], 'val': []}

    for epoch in range(1, epochs + 1):
        # Train phase
        model.train()
        train_loss = 0.0
        for Xb, Yb in tr_dl:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lam)

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item() * Xb.size(0)
        train_loss /= len(tr_dl.dataset)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for Xv, Yv in va_dl:
                Xv, Yv = Xv.to(device), Yv.to(device)
                pred = model(Xv)
                loss, _, _ = total_loss(pred, Yv, Xv, stats, lam)
                val_loss += loss.item() * Xv.size(0)
        val_loss /= len(va_dl.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4e} | Val Loss: {val_loss:.4e}")

        if val_loss < best_val - 1e-6:
            best_val = val_loss
            wait = 0
            torch.save(model.state_dict(), "best_omn_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load("best_omn_ai.pth", map_location=device))
    return history


# -----------------------------------------------------------------------------
# 6. Visualization Helpers
# -----------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def plot_parity(model, dataset, stats, device):
    X = torch.from_numpy(dataset.X).to(device)
    with torch.no_grad():
        preds = model(X).cpu().numpy()

    Y_true = dataset.Y * dataset.Y_std + dataset.Y_mean
    Y_pred = preds * dataset.Y_std + dataset.Y_mean

    plt.figure(figsize=(6, 6))
    plt.scatter(Y_true.ravel(), Y_pred.ravel(), s=4, alpha=0.5)
    m, M = Y_true.min(), Y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    plt.show()


def plot_uncertainty(model, dataset, stats, device):
    X = torch.from_numpy(dataset.X).to(device)
    _, std = mc_dropout_predict(model, X, T=50)
    u = std[:, 0].cpu().numpy()
    plt.figure()
    plt.hist(u, bins=30, color='purple')
    plt.xlabel('Std of FR predictions')
    plt.ylabel('Frequency')
    plt.show()


# -----------------------------------------------------------------------------
# 7. Main Execution
# -----------------------------------------------------------------------------
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # prepare data & stats (cast stats to float32)
    ds = OmniversalDataset(n_samples=5000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std':  torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  dtype=torch.float32, device=device)
    }

    # train/validation split
    n_val = int(len(ds) * 0.2)
    tr_ds, va_ds = random_split(ds, [len(ds) - n_val, n_val])
    tr_dl = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=256, shuffle=False)

    # initialize, train, visualize
    model = OmniversalAI()
    history = train(
        model, tr_dl, va_dl, stats, device,
        lr=1e-3, weight_decay=1e-5, lam=1.0,
        epochs=100, patience=10
    )

    plot_history(history)
    plot_parity(model, ds, stats, device)
    plot_uncertainty(model, ds, stats, device)